# Root finding

Per relazione: fai un confronto fra le radici ottenute qui e in secante, così come i valori di q e di C

In [ ]:
using Plots
using ForwardDiff
using Symbolics
using Markdown
using LaTeXStrings
using Printf
include("C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\interpolation.jl")
include("C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\librerie\\non_linear_roots.jl")
function autosave(fig, filename)
    path = "C:\\ALL\\Stefano\\Bicocca\\3terzo_anno\\lab_comp\\lab_computazionale1\\relazione\\immagini"
    savefig(fig, joinpath(path, filename))
end

**(a)** Rewrite the equations into the standard form for rootfinding, $f(x) = 0$, and compute $f'(x)$.

In [ ]:
# Function cell
f1(x) = x^2 - exp(-x)
f2(x) = 2x - tan(x)
f3(x) = exp(x+1) - 2 - x
df1(x) = ForwardDiff.derivative(f1, x)
df2(x) = ForwardDiff.derivative(f2, x)
df3(x) = ForwardDiff.derivative(f3, x)
d2f1(x) = ForwardDiff.derivative(df1, x)
d2f2(x) = ForwardDiff.derivative(df2, x)
d2f3(x) = ForwardDiff.derivative(df3, x)

#For ulterior studies:
d3f2(x) = ForwardDiff.derivative(d2f2, x)

@variables x
sym_f1 = x^2 - exp(-x)
sym_f2 = 2x - tan(x)
sym_f3 = exp(x+1) - 2 - x
sym_df1 = Symbolics.derivative(sym_f1, x)
sym_df2 = Symbolics.derivative(sym_f2, x)
sym_df3 = Symbolics.derivative(sym_f3, x)
sym_d2f1 = Symbolics.derivative(sym_df1, x)
sym_d2f2 = Symbolics.derivative(sym_df2, x)
sym_d2f3 = Symbolics.derivative(sym_df3, x)

display(Markdown.parse("First derivative"))
display(sym_df1)
display(sym_df2)
display(sym_df3)
display(Markdown.parse("Second derivative"))
display(sym_d2f1)
display(sym_d2f2)
display(sym_d2f3)
colors = colors = ["#0F2080",  # blu scuro
           "#F5793A",  # arancione
           "#A95AA1",  # viola
           "#85C0F9"]  # azzurro

**(b)** Make a plot of $f$ over the given interval and determine how many roots lie in the interval. \
**(c)** Use Newton's method to find each root.\
**(d)** Study the error in the Newton sequence to determine numerically whether the convergence is roughly quadratic for the given root.

In general, defining $ \epsilon_n = \log(x_n-r)$ we have 
$$
    \epsilon_{n+1}\approx q\,\epsilon_{n} -\log|C|
    \quad
    \Rightarrow
    \quad
    {\epsilon_{n+1}\over \epsilon_n}\approx q -{\log|C|\over \epsilon_n}\ \overset{n\to\infty}{\longrightarrow}\ q
$$
similarly as in 3_2_2.ipynb. \
After finding q, we can compute $C$ with the following formula (actually definition): 
$$
    \lim_{n\to\infty} {|x_{n+1}-r|\over|x_n-r|^q}=C\,,
$$
just to compare it with the specific $C$ formula of Newton method:
$$
  C={1\over 2}\bigg|{f''(r)\over f'(r)}\bigg|\,.
$$

### $x^2 - e^{-x} = 0$, over $[-2,2]$

In [ ]:
x1 = [i for i in -2:0.1:2]
y1 = f1.(x1) 

fig1 = plot(figsize=(800, 600),
            xlabel=L"x", ylabel=L"y",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )

plot!(fig1, x1, y1,
      label = L"$x^2 - e^{-x}$",
      color = colors[1],
     )

plot!(fig1, x1, zeros(length(x1)),
      label = "",
      color = colors[2],
     )

autosave(fig1, "3321.pdf")
display(fig1)

In [ ]:
first_guess = -2
zero1, step1 = newt_met_steps(f1, first_guess, 1)
result_text = @sprintf("Estimated root value: %.3f", zero1)
display(Markdown.parse(result_text))
result_text = @sprintf("Function value in estimated root: %.3f", f1(zero1))
display(Markdown.parse(result_text))
display(Markdown.parse("Number of steps: $(length(step1))"))

Now we study the errors

Fai un grafico su anche su q

In [ ]:
eps = log.(abs.(step1 .- zero1))
en = eps[1:end-2]       #errori ϵ_n
en1 = eps[2:end-1]      #errori ϵ_{n+1}
q_asympt = en1 ./ en
#We print the last one because it is an asymptotical study
result_text = @sprintf("q evaluated asymptotically: %.3f", q_asympt[end])
display(Markdown.parse(result_text))

fig1_0 = plot(figsize=(800, 600),
            xlabel=L"n", ylabel=L"d_{n+1} / d_n",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )
plot!(fig1_0, 1:length(q_asympt), q_asympt,
        label = L"q_n",
        color = colors[1],
        marker = :circle,
         )
plot!(fig1_0, 1:length(q_asympt), 2 .*ones(length(q_asympt)),
        label = L"q_{exact}",
        color = colors[2],
        )

autosave(fig1_0, "3321_q.pdf")
display(fig1_0)

Attento ai nomi, riguarda se C si chiama convergence rate e non sia q a chiamarsi così

In [ ]:
#d is the sequence of error ratio
d1 = [abs(step1[i+1]-zero1)/abs(step1[i]-zero1)^2 for i in 1:(length(step1)-2)]        #Iteration till length(step1)-2 because I don't want include the root.
k1=[i for i in 1:1:(length(step1)-2)]
#C expected value:
C1 = 0.5*abs(d2f1(zero1)/df1(zero1))
result_text = @sprintf("C evaluated asymptotically =  %.3f", d1[end])
display(Markdown.parse(result_text))
result_text = @sprintf("C expected = %.3f", C1)
display(Markdown.parse(result_text))

fig1_1 = plot(xlabel=L"k", ylabel=L"|ϵ_{k+1}|/|ϵ_{k}|^2",
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:bottomright
             )

plot!(fig1_1, k1, d1,
     label=L"C_k",
     color=colors[1],
     marker=:circle,
     markersize=3.5,
     )

x_step = [i for i in k1[1]:0.1:k1[end]]
plot!(fig1_1, x_step, ones(length(x_step))*C1,
     label=L"C",
     color=colors[2],
     )

autosave(fig1_1, "3321_C.pdf")
display(fig1_1)

### $2x - \tan x=0 $, over $[-0.2,1.4]$

In [ ]:
x2 = [i for i in -0.2:0.01:1.4]
y2 = f2.(x2) 

fig2 = plot(figsize=(800, 600),
            xlabel=L"x", ylabel=L"y",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )

plot!(fig2, x2, y2,
      label = L"2x - tan{x}",
      color = colors[1],
      )

plot!(fig2, x2, zeros(length(x2)),
      label = "",
      color = colors[2],
     )

autosave(fig2, "3322.pdf")
display(fig2)

In [ ]:
first_guess_1 = 0.65
zero2_1, step2_1 = newt_met_steps(f2, first_guess_1, 1)
display(Markdown.parse("I expect the root and function' values to be the same because the tangent near 0 behaves like x, so f behaves like 2x - x, which is x."))
result_text = @sprintf("First root estimated value: %.3e", zero2_1)
display(Markdown.parse(result_text))
result_text = @sprintf("Function value in first estimated root: %.3e", f2(zero2_1))
display(Markdown.parse(result_text))
display(Markdown.parse("Number of steps: $(length(step2_1))"))

first_guess_2 = 1.4
zero2_2, step2_2 = newt_met_steps(f2, first_guess_2, 1)
result_text = @sprintf("Second root estimated value: %.3f", zero2_2)
display(Markdown.parse(result_text))
result_text = @sprintf("Function value in second estimated root: %.2e", f2(zero2_2))
display(Markdown.parse(result_text))
display(Markdown.parse("Number of steps: $(length(step2_1))"))

Now we study the errors. 
 
The second root doesn't give any kind of problem. 

The first root, however, is quite interesting: asymptotically, we obtain a q value of 3.042, which is not what we expected.

In [ ]:
#First root
eps = log.(abs.(step2_1 .- zero2_1))
en = eps[1:end-2]       #errori ϵ_n
en1 = eps[2:end-1]      #errori ϵ_{n+1}
q_asympt1 = en1 ./ en
#We print the last one because it's an asymptotical study
result_text = @sprintf("q value for r = %.1f: %.3f", zero2_1, q_asympt1[end])
display(Markdown.parse(result_text))

fig2_00 = plot(figsize=(800, 600),
            xlabel=L"n", ylabel=L"d_{n+1} / d_n",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )

plot!(fig2_00, 1:length(q_asympt1), q_asympt1,
        label = L"q_n",
        color = colors[1],
        marker = :circle,
         )
plot!(fig2_00, 1:length(q_asympt1), 3 .*ones(length(q_asympt1)),
        label = L"q_{expected}",
        color = colors[2],
        )
autosave(fig2_00, "3322_q1.pdf")
display(fig2_00)


#second root
eps = log.(abs.(step2_2 .- zero2_2))
en = eps[1:end-2]       #errori ϵ_n
en1 = eps[2:end-1]      #errori ϵ_{n+1}
q_asympt2 = en1 ./ en
#We print the last one because it's an asymptotical study
result_text = @sprintf("q value for r = %.1f: %.3f", zero2_2, q_asympt2[end])
display(Markdown.parse(result_text))

fig2_01 = plot(figsize=(800, 600),
            xlabel=L"n", ylabel=L"d_{n+1} / d_n",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )

plot!(fig2_01, 1:length(q_asympt2), q_asympt2,
        label = L"q_n",
        color = colors[1],
        marker = :circle,
         )
plot!(fig2_01, 1:length(q_asympt2), 2 .*ones(length(q_asympt2)),
        label = L"q_{exact}",
        color = colors[2],
        )

autosave(fig2_01, "3322_q2.pdf")
display(fig2_01)

For first root there is a slight problem: second derivative is zero, so Newton's formula for C is not valid. \
We must develop the Taylor series till third order. Sums are in section pen_and_paper, but result is:
$$
\frac{\epsilon_{k+1}}{\epsilon_k^3} = \frac{ f'''(r)}{3 f'(r)} \quad \text{and} \quad C = \frac{ f'''(r)}{3 f'(r)}
$$ 

Dal grafico non si capisce a che altezza sia la costante asintotica

In [ ]:
#d is the sequence of error ratio
d2_1 = [abs(step2_1[i+1]-zero2_1)/abs(step2_1[i]-zero2_1)^3 for i in 1:(length(step2_1)-2)]        #Iteration till length(step2)-2 because I don't want include the root.
k2_1=[i for i in 1:1:(length(step2_1)-2)]
C2 = (d3f2(zero2_1)/df2(zero2_1))/(-3)
result_text = @sprintf("C evaluated asymptotically =  %.7f", d2_1[end])
display(Markdown.parse(result_text))
result_text = @sprintf("C expected by Taylor expansion till third order =  %.7f", C2)
display(Markdown.parse(result_text))

fig2_1 = plot(xlabel=L"k", ylabel=L"|ϵ_{k+1}|/|ϵ_{k}|^3",
              yticks = 0.6:0.2:2.4,
              yrange=(0.6, 2.5),
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:topright,
             )

plot!(fig2_1, k2_1, d2_1,
     label=L"C_k",
     marker=:circle,
     markersize=3.5,
     color = colors[1],
     )

x_step = [i for i in k2_1[1]:0.1:k2_1[end]]
plot!(fig2_1, x_step, ones(length(x_step))*C2,
     label=L"C",
     color = colors[2],
     )

autosave(fig2_1, "3322_C1.pdf")
display(fig2_1)

We shall see what happens when we force $q = 2$ for first root. We expect it to be zero because of its second derivative.

In [ ]:
#d is the sequence of error ratio
d2_1 = [abs(step2_1[i+1]-zero2_1)/abs(step2_1[i]-zero2_1)^2 for i in 1:(length(step2_1)-2)]        #Iteration till length(step2)-2 because I don't want include the root.
k2_1=[i for i in 1:1:(length(step2_1)-2)]
C2_1 = 0.5*abs(d2f2(zero2_1)/df2(zero2_1))
result_text = @sprintf("C expected = %.3f", C2_1)
display(Markdown.parse(result_text))
result_text = @sprintf("C evaluated asymptotically =  %.3f", d2_1[end])
display(Markdown.parse(result_text))

fig2_1 = plot(xlabel="Steps", ylabel="Convergence",
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:topright
             )

plot!(fig2_1, k2_1, d2_1,
     label=L"\frac{|ϵ_{k+1}|}{|ϵ_{k}|^2}",
     marker=:circle,
     markersize=3.5,
     color = colors[1],
     )

x_step = [i for i in k2_1[1]:0.1:k2_1[end]]
plot!(fig2_1, x_step, ones(length(x_step))*C2_1,
     label=L"C",
     color = colors[2],
     )

display(fig2_1)

For the second root we expect normal behaviour, $q = 2$ and non zero convergence for $\frac{|\epsilon_{k+1}|}{|\epsilon_k|^2} $

In [ ]:
#d is the sequence of error ratio
d2_2 = [abs(step2_2[i+1]-zero2_2)/(step2_2[i]-zero2_2)^2 for i in 1:(length(step2_2)-2)]        #Iteration till length(step2)-2 because I don't want include the root.
k2_2=[i for i in 1:1:(length(step2_2)-2)]
#C expected value:
C2_2 = 0.5*abs(d2f2(zero2_2)/df2(zero2_2))
display(Markdown.parse("Asymptotic error constant expected = $C2_2"))
result_text = @sprintf("C evaluated asymptotically =  %.3f", d2_2[end])
display(Markdown.parse(result_text))

fig2_2 = plot(xlabel=L"k", ylabel=L"|ϵ_{k+1}|/|ϵ_{k}|^2",
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:bottomright
             )

plot!(fig2_2, k2_2, d2_2,
     label=L"C_k",
     color=colors[1],
     marker=:circle,
     markersize=3.5,
     )

x_step = [i for i in k2_2[1]:0.1:k2_2[end]]
plot!(fig2_2, x_step, ones(length(x_step))*C2_2,
     label=L"C",
     color=colors[2],
     )

autosave(fig2_2, "3322_C2.pdf")
display(fig2_2)

### $e^{x+1} -2 - x = 0 $, over $[-2,2]$

In [ ]:
x3 = [i for i in -2:0.1:2]
y3 = f3.(x3) 

fig3 = plot(figsize=(800, 600),
            xlabel=L"x", ylabel=L"y",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )

plot!(fig3, x3, y3,
      label = L"e^{x+1} - 2 - x",
      color = colors[1],
     )

plot!(fig3, x3, zeros(length(x3)),
      label = "",
      color = colors[2],
      )

autosave(fig3, "3323.pdf")
display(fig3)

In [ ]:
first_guess = 2
zero3, step3 = newt_met_steps(f3, first_guess, 1)
result_text = @sprintf("Estimated root value: %.3f", zero3)
display(Markdown.parse(result_text))
result_text = @sprintf("Function value in estimated root: %.3f", f3(zero3))
display(Markdown.parse(result_text))
display(Markdown.parse("Number of steps: $(length(step3))"))

For q value we now expect it to be around 1 because of its multiplicity

In [ ]:
eps = log.(abs.(step3 .- zero3))
en = eps[1:end-2]       #errori ϵ_n
en1 = eps[2:end-1]      #errori ϵ_{n+1}
q_asympt = en1 ./ en
#We print the last one because it's an asymptotical study
result_text = @sprintf("q value for r = %.1f: %.3f", zero3, q_asympt[end])
display(Markdown.parse(result_text))

fig3_0 = plot(figsize=(800, 600),
            xlabel=L"n", ylabel=L"d_{n+1} / d_n",
            framestyle=:box,
            grid=true, gridalpha=0.5,
            )
plot!(fig3_0, 1:length(q_asympt), q_asympt,
        label = L"q_n",
        color = colors[1],
        marker = :circle,
         )
plot!(fig3_0, 1:length(q_asympt), ones(length(q_asympt)),
        label = L"q_{exact}",
        color = colors[2],
        )

autosave(fig3_0, "3323_q.pdf")
display(fig3_0)

Below we see what happens if we don't care about root multiplicity.

In [ ]:
#d is the sequence of error ratio
d3 = [abs(step3[i+1]-zero3)/(step3[i]-zero3)^2 for i in 1:(length(step3)-2)]        #Iteration till length(step2)-2 because I don't want include the root.
k3=[i for i in 1:1:(length(step3)-2)]
#C expected value:
C3 = 0.5*abs(d2f3(zero3)/df3(zero3))
result_text = @sprintf("C evaluated asymptotically =  %.3e", d3[end])
display(Markdown.parse(result_text))
result_text = @sprintf("C expected = %.3e", C3)
display(Markdown.parse(result_text))
fig3_1 = plot(xlabel=L"k", ylabel=L"|ϵ_{k+1}|/|ϵ_{k}|^2",
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:topright,
              yticks = [i*10.0^6 for i in 3:3:15],
              yformatter = (x -> @sprintf("%.1e", x)),
             )
plot!(fig3_1, k3, d3,
     label=L"C_k",
     marker=:circle,
     markersize=3.5,
     color = colors[1],
     )

x_step = [i for i in k3[1]:0.1:k3[end]]
plot!(fig3_1, x_step, ones(length(x_step))*C3,
     label=L"C",
     color = colors[2],
     )

autosave(fig3_1, "3323_C1.pdf")
display(fig3_1)

By looking at the last graph, I notice that the succession doesn't converge where I think it should, so I must have forgotten multiplicity. To find out the zero's multiplicity I can use this relation: $ \epsilon_{k+1} = (1-{1\over m})\epsilon_k $, where m is the root's multiplicity and $C=(1-{1\over m})$ \
From the plot below we can notice that, after 20th step or so, C evaluation isn't accurate anymore because occurs a numerical phenomena, probably due to division of small errors. For our m guess we'll use C of the 15th step, where convergence is good.


In [ ]:
#d is the sequence of error ratio
d3 = [abs(step3[i+1]-zero3)/abs(step3[i]-zero3)^1 for i in 1:(length(step3)-2)]        #Iteration till length(step2)-2 because I don't want include the root.
k3=[i for i in 1:1:(length(step3)-2)]

#plot section
fig3_2 = plot(xlabel=L"k", ylabel=L"|ϵ_{k+1}|/|ϵ_{k}|",
              figsize=(800, 600),
              framestyle=:box,
              grid=true, gridalpha=0.5,
              legend=:topright
             )
plot!(fig3_2, k3, d3,
     label=L"C_k",
     marker=:circle,
     markersize=3.5,
     color = colors[1],
     )

x_step = [i for i in k3[1]:0.1:k3[end]]
plot!(fig3_2, x_step, ones(length(x_step))*0.5,
     label=L"C",
     color = colors[2],
     )
autosave(fig3_2, "3323_C2.pdf")
display(fig3_2)

C_asympt = d3[end-10]
m = 1/(1-C_asympt)
result_text = @sprintf("C evaluated asymptotically =  %.3f", C_asympt)
display(Markdown.parse(result_text))
result_text = @sprintf("Multiplicity value for r = %.1f: %.3f", zero3, m)
display(Markdown.parse(result_text))
